### This notebook scrapes data tables from a PDF file

In [1]:
import requests
import urllib3
import os
import pandas as pd
import numpy as np
import pdfplumber 
import io


os.chdir(r'/Users/swilson/Library/CloudStorage/OneDrive-Personal/python/python notebooks/GitHub/Scraping PDF')
cwd = os.getcwd()
print("Current working directory is:", cwd)

Current working directory is: /Users/swilson/Library/CloudStorage/OneDrive-Personal/python/python notebooks/GitHub/Scraping PDF


In [2]:
#function to see the names of the tables

def read_pdf(url: str) -> str:
    http = urllib3.PoolManager()
    temp = io.BytesIO()
    temp.write(http.request("GET", url).data)
    with pdfplumber.open(temp) as pdf:
        page = pdf.pages[20]
        tables = page.extract_tables()
    return tables


In [3]:
#read in pdf from DOD's Per- and polyfluoroalkyl substances (PFAS) website that contains PFAS concentration data
url = 'https://www.acq.osd.mil/eie/eer/ecc/pfas/docs/reports/FY18-HASC-Brief-on-PFOS-PFOA_Mar2018.pdf'

tables = read_pdf(url)

In [4]:
print(tables)

[[['Summary of Testing for DoD Drinking Water Systems and Actions Taken where Results were Above the EPA LHAs1', None, None, None, None, None, None, None, None, None, None, None, None], ['DoD\nComponent', 'Installation', 'State/\nTerritory/\nOverseas\nLocation', 'DoD Purveyor Drinking Water Systems', None, None, None, None, 'Non-DoD Purveyor Drinking Water Systems', None, None, None, None], [None, None, None, 'Total\nNumber\nTested', 'Number\nthat\nTested\nabove the\nEPA\nLHAs1', 'Results (PFOS/ PFOA) or Range\nabove EPA LHAs (ppt)', 'Actions Taken as of August 31, 20172', 'Actions Planned after August 31,\n2017, and Timeline to Implement the\nActions to Reduce PFOS/PFOA\nBelow LHA', 'Total\nNumber\nTested', 'Number\nthat\nTested\nabove the\nEPA\nLHAs1', 'Results\n(PFOS/\nPFOA) or\nRange\nabove EPA\nLHAs (ppt)', 'Actions Taken as of August 31, 20172', 'Actions Planned after August 31,\n2017, and Timeline to Implement the\nActions to Reduce PFOS/PFOA\nBelow LHA'], ['Army', 'JB Lewis-McC

In [5]:
#function to extract tables from pdf

def extract_tables(url: str) -> str:
    http = urllib3.PoolManager()
    temp = io.BytesIO()
    temp.write(http.request("GET", url).data)
    with pdfplumber.open(temp) as pdf:
          lst = [p.extract_table() for p in pdf.pages]
    return lst

In [6]:
lst = extract_tables(url)

In [7]:
lst[:15]

[None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 [['Component',
   'Total Installations with\nknown or suspected release of\nPFOS/PFOA (as of August\n31, 2017)',
   'Number of Installations\nSampled where results\nexceeded EPA LHA (as of\nAugust 31, 2017)',
   'Total number of\ngroundwater wells\nsampled',
   'Number of\ngroundwater\nwells that\ntested above\nthe EPA\nLHA'],
  ['Army', '64', '9', '258', '104'],
  ['Navy/USMC', '127', '40', '1,368', '784'],
  ['Air Force', '203', '39', '1,022', '719'],
  ['DLA', '7', '2', '20', '14'],
  ['Total', '401', '90', '2,668', '1,621']],
 None,
 None,
 None,
 None,
 None]

In [8]:
# remove None values and view first few sublists
clean = [x for x in lst if x != None]

clean[:2]

[[['Component',
   'Total Installations with\nknown or suspected release of\nPFOS/PFOA (as of August\n31, 2017)',
   'Number of Installations\nSampled where results\nexceeded EPA LHA (as of\nAugust 31, 2017)',
   'Total number of\ngroundwater wells\nsampled',
   'Number of\ngroundwater\nwells that\ntested above\nthe EPA\nLHA'],
  ['Army', '64', '9', '258', '104'],
  ['Navy/USMC', '127', '40', '1,368', '784'],
  ['Air Force', '203', '39', '1,022', '719'],
  ['DLA', '7', '2', '20', '14'],
  ['Total', '401', '90', '2,668', '1,621']],
 [['Poly- and per- fluorinated alkyl substances (PFASs)',
   None,
   None,
   None,
   None],
  ['Perfluoroalkyl substances',
   None,
   'Polyfluoroalkyl substances',
   None,
   None],
  ['Perfluoroalkyl acids\n(PFAAs)\n(',
   'Sulfonamide- containing\nprecursors\ne.g., FOSA, N-EtFOSE)',
   None,
   None,
   'Fluorotelomer\nprecursors\n(e.g., 6:2 FtS)'],
  ['PFSAs\n(e.g., PFOS)', '', None, '', None]]]

In [9]:
#flatten sublist structure
flat_list = [item for sublist in clean for item in sublist]
flat_list[:8]

[['Component',
  'Total Installations with\nknown or suspected release of\nPFOS/PFOA (as of August\n31, 2017)',
  'Number of Installations\nSampled where results\nexceeded EPA LHA (as of\nAugust 31, 2017)',
  'Total number of\ngroundwater wells\nsampled',
  'Number of\ngroundwater\nwells that\ntested above\nthe EPA\nLHA'],
 ['Army', '64', '9', '258', '104'],
 ['Navy/USMC', '127', '40', '1,368', '784'],
 ['Air Force', '203', '39', '1,022', '719'],
 ['DLA', '7', '2', '20', '14'],
 ['Total', '401', '90', '2,668', '1,621'],
 ['Poly- and per- fluorinated alkyl substances (PFASs)',
  None,
  None,
  None,
  None],
 ['Perfluoroalkyl substances', None, 'Polyfluoroalkyl substances', None, None]]

In [10]:
#convert flatten list to pandas dataframe
df = pd.DataFrame(flat_list)
df.columns = df.iloc[0]
df.head(10)

,Component,"Total Installations with\nknown or suspected release of\nPFOS/PFOA (as of August\n31, 2017)","Number of Installations\nSampled where results\nexceeded EPA LHA (as of\nAugust 31, 2017)",Total number of\ngroundwater wells\nsampled,Number of\ngroundwater\nwells that\ntested above\nthe EPA\nLHA,None,None,None,None,None,None,None,None,None,None
0,Component,Total Installations with\nknown or suspected r...,Number of Installations\nSampled where results...,Total number of\ngroundwater wells\nsampled,Number of\ngroundwater\nwells that\ntested abo...,None,None,None,None,None,None,None,None,None,None
1,Army,64,9,258,104,None,None,None,None,None,None,None,None,None,None
2,Navy/USMC,127,40,"1,368",784,None,None,None,None,None,None,None,None,None,None
3,Air Force,203,39,"1,022",719,None,None,None,None,None,None,None,None,None,None
4,DLA,7,2,20,14,None,None,None,None,None,None,None,None,None,None
5,Total,401,90,"2,668","1,621",None,None,None,None,None,None,None,None,None,None
6,Poly- and per- fluorinated alkyl substances (P...,None,None,None,None,None,None,None,None,None,None,None,None,None,None
7,Perfluoroalkyl substances,None,Polyfluoroalkyl substances,None,None,None,None,None,None,None,None,None,None,None,None
8,Perfluoroalkyl acids\n(PFAAs)\n(,"Sulfonamide- containing\nprecursors\ne.g., FOS...",None,None,"Fluorotelomer\nprecursors\n(e.g., 6:2 FtS)",None,None,None,None,None,None,None,None,None,None
9,"PFSAs\n(e.g., PFOS)",,None,,None,None,None,None,None,None,None,None,None,None,None


In [11]:
#break off the first nine rows to pull out summary table of number of instllations with PFAS from page 9 of the pdf
summary = df.loc[0:9,:]
summary.columns = summary.columns.str.replace(r'\n',' ', regex=True) #remove n lines
summary = summary.drop(columns=[None])
summary = summary.iloc[:-4]
summary = summary.iloc[1:]

summary = summary.reset_index(drop=True)

summary

,Component,"Total Installations with known or suspected release of PFOS/PFOA (as of August 31, 2017)","Number of Installations Sampled where results exceeded EPA LHA (as of August 31, 2017)",Total number of groundwater wells sampled,Number of groundwater wells that tested above the EPA LHA
0,Army,64,9,258,104
1,Navy/USMC,127,40,"1,368",784
2,Air Force,203,39,"1,022",719
3,DLA,7,2,20,14
4,Total,401,90,"2,668","1,621"


In [12]:
#next table with total number of installations and drinking water systems per military branch
df_no_summary = df.iloc[10:]
df_no_summary = df_no_summary.reset_index()
df_summary_installations = df_no_summary.iloc[:9]
df_summary_installations = df_summary_installations.reset_index()
df_summary_installations = df_summary_installations.drop(columns=["level_0", "index"])

#exclude empty columns
df_summary_installations = df_summary_installations.iloc[:,:-7]

#make none type nan and forward fill na values
df_summary_installations = df_summary_installations.fillna(value=np.nan)
df_summary_installations = df_summary_installations.ffill(axis=1)#, 'columns')


#remove citation numbers from rows that will become headers
df_summary_installations.loc[1] = df_summary_installations.loc[1].str.replace("[0-9]", "", regex=True)
df_summary_installations.loc[2] = df_summary_installations.loc[2].str.replace("[0-9]", "", regex=True)

#assign two rows as headers
df_summary_installations = df_summary_installations.replace(r'\n',' ', regex=True) #remove n lines
df_summary_installations.columns=pd.MultiIndex.from_arrays(df_summary_installations.iloc[1:3].values)
df_summary_installations = df_summary_installations.iloc[3:]

df_summary_installations = df_summary_installations.reset_index(drop=True)

df_summary_installations

DoD Component Number of Installations in US/Territories/ Overseas  \
            NaN                                                 NaN   
0          Army                                              2,905    
1          Navy                                                108    
2          USMC                                                 68    
3     Air Force                                                177    
4           DLA                                                  8    
5    DoD Totals                                              3,266    

  Number of DoD Drinking Water Systems for DoD Installations                   \
                                                DoD Purveyor Non-DoD Purveyor   
0                                                255                    2,738   
1                                                100                      236   
2                                                 28                       52   
3                                                140                      134   
4                                                  1                        7   
5                                                524                    3,167   

  Number of DoD Drinking Water Systems Tested by Either DoD or the non- DoD Purveyor  \
                                                                        DoD Purveyor   
0                                                255                                   
1                                                100                                   
2                                                 28                                   
3                                                140                                   
4                                                  1                                   
5                                                524                                   

                     \
  Non-DoD Purveyor    
0             1,589   
1               236   
2                52   
3               134   
4                 7   
5             2,018   

  Number of DoD Drinking Water Systems Tested Where PFOS/PFOA > EPA LHA  \
                                                           DoD Purveyor   
0                                                 12                      
1                                                  5                      
2                                                  1                      
3                                                  6                      
4                                                  0                      
5                                                 24                      

                    
  Non-DoD Purveyor  
0                4  
1                1  
2                2  
3                5  
4                0  
5               12

In [26]:
#next table with PFAS concentrations in drinking water for Army bases


df_army_dod_installation = df.iloc[19:] 
df_army_dod_installation.reset_index(drop=True, inplace=True)

df_army_dod_installation=df_army_dod_installation.copy()

# # #make none type nan and forward fill na values loc[:, ['A']]
df_army_dod_installation.loc[0] = df_army_dod_installation.loc[0].fillna(value=np.nan).copy(deep=True)
df_army_dod_installation.loc[0] = df_army_dod_installation.loc[0].fillna(method="ffill")
df_army_dod_installation.loc[1] = df_army_dod_installation.loc[1].fillna(value=np.nan)
df_army_dod_installation.loc[1] = df_army_dod_installation.loc[1].fillna(method="ffill")

#remove excess numbers from rows that will become headers
df_army_dod_installation.loc[0] = df_army_dod_installation.loc[0].str.replace("[0-9]", "", regex=True)
df_army_dod_installation.loc[1] = df_army_dod_installation.loc[1].str.replace("[0-9]", "",regex=True)
df_army_dod_installation = df_army_dod_installation.replace(r'\n',' ', regex=True) #remove n lines
df_army_dod_installation.columns=pd.MultiIndex.from_arrays(df_army_dod_installation.iloc[0:3].values)

#drop off row with column names and only select rows relevant to Army DOD installations
df_army_dod_installation = df_army_dod_installation.iloc[3:25]
df_army_dod_installation.reset_index(drop=True, inplace=True)

#drop rows with extra column names
df_army_dod_installation.iloc[:,0] = df_army_dod_installation.iloc[:,0].fillna(value=np.nan)
df_army_dod_installation = df_army_dod_installation[df_army_dod_installation.iloc[:,0].notna()]
df_army_dod_installation =  df_army_dod_installation[df_army_dod_installation.iloc[:,0].str.contains("Army")]
df_army_dod_installation.reset_index(drop=True, inplace=True)

#exclude empty columns
df_army_dod_installation = df_army_dod_installation.iloc[:,:-2]


df_army_dod_installation

Summary of Testing for DoD Drinking Water Systems and Actions Taken where Results were Above the EPA LHAs  \
                                                                                               DoD Component   
                                                                                                         NaN   
0                                                Army                                                          
1                                                Army                                                          
2                                                Army                                                          
3                                                Army                                                          
4                                                Army                                                          
5                                                Army                                                          
6                                                Army                                                          
7                                                Army                                                          
8                                                Army                                                          
9                                                Army                                                          
10                                               Army                                                          
11                                               Army                                                          
12                                               Army                                                          

                                             \
                               Installation   
                                        NaN   
0             81st RSC: E. Earle Rives AFRC   
1       99th RSC Martinsburg Memorial USARC   
2                       Fort Hunter Liggett   
3                           Ft. Leavenworth   
4   JB Lewis-McChord: Fort Lewis Cantonment   
5                          Soto Cano AB, HN   
6               USAG Daegu, KR Camp Carroll   
7                USAG Daegu, KR Camp Walker   
8        USAG Red Cloud, KR: Camp Red Cloud   
9          USAG Red Cloud, KR: Camp Stanley   
10      USAG Benelux - Caserne Daumerie, BE   
11                           Belmont Armory   
12                                 El Campo   

                                                                            \
   State/ Territory/ Overseas Location DoD Purveyor Drinking Water Systems   
                                   NaN                 Total Number Tested   
0                       North Carolina                                       
1                        West Virginia                                       
2                           California                                   1   
3                               Kansas                                       
4                           Washington                                   4   
5                             Honduras                                   1   
6                                Korea                                   1   
7                                Korea                                   1   
8                                Korea                                   1   
9                                Korea                                   1   
10                             Belgium                                       
11                            Michigan                                   1   
12                               Texas                                   1   

                                           \
                                            
   Number that Tested above the EPA LHAs1   
0                                           
1                           